# SCF versus direct minimisation

In this notebook we want to numerically investigate the convergence of SCF and direct minimisation as the gap of a system closes.

<!--- Write your own DM algo (i.e. not optim) -->

We will model this by considering a standard silicon lattice and increasing the lattice constant. We will use the following setup with increasing $a$:

In [ ]:
using DFTK

function silicon_setup(a=10.26; Ecut=15.0, kgrid=[1, 1, 1])
    lattice = a / 2 * [[0 1 1.];
                       [1 0 1.];
                       [1 1 0.]]
    
    # Calculation parameters
    Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
    atoms     = [Si, Si]
    positions = [ones(3)/8, -ones(3)/8]
    
    # Construct the model
    model = model_LDA(lattice, atoms, positions)
    PlaneWaveBasis(model; Ecut, kgrid)
end

**Exercise:**   
(a) Run a damped SCF (without preconditioning) for $a = 10.5$ and $a = 11.4$. For both cases print the band gap. For this case with only one $k$-point is exactly the difference between the 5th (lowest unoccupied) and 4th (highest occupied) eigenvalue. You can od this as follows:

```julia
a = 10.5
scfres = self_consistent_field(silicon_setup(a); is_converged=DFTK.ScfConvergenceDensity(1e-7), mixing=SimpleMixing(), damping=0.8);
@show scfres.eigenvalues[1][5] - scfres.eigenvalues[1][4]
```

In [ ]:
a = 10.5
scfres = self_consistent_field(silicon_setup(a); is_converged=DFTK.ScfConvergenceDensity(1e-7), mixing=SimpleMixing(), damping=0.8);
@show scfres.eigenvalues[1][5] - scfres.eigenvalues[1][4]

(b) Compare the behaviour with direct minimisation for both values of the lattice constant $a$. You can run DM as follows:
```julia
using Optim

a = 10.5
dmres = direct_minimization(silicon_setup(a); tol=1e-7);
```